## Across race prediction - CV2

In [6]:
.libPaths(c('/anaconda2/lib/R/library',.libPaths()))

library(rrBLUP)
library(ggplot2)
library(BGLR)

my.read.vcf <- function(file, special.char="##", ...) {
  my.search.term <- paste0(special.char, ".*")  # Making a search term that looks like: "##.*", tells R to find anything containing the pattern "##" followed by anything (* is wildcard)
  clean.lines <- sub(my.search.term, "", readLines(file)) # Replace any line containing the search term with nothing (in other words remove it)
  clean.lines2 <- sub("#CHROM", "CHROM", clean.lines) # Replace the #CHROM term in the header with CHROM, so R doesn't treat it as a special character
  read.table(..., text=paste(clean.lines2, collapse="\n")) # Pass the cleaned up lines to read.table
}

### make CV fold dataframe for each race five equal folds with sum total to ~ number of individuals in that race

In [31]:
setwd("/Users/ssapkot/Documents/Experiments/SAP_GS_PopStr")
Y <- read.csv('data/BLUEs_pheno_all.csv', header=T, row.names = 1)
Y$SN = 1:nrow(Y)


In [45]:
##Setting up a loop for running for each cluster
for (a in 1:5){
    
    Y1 = Y[Y$Cluster == a,]
    nr = floor(nrow(Y1)/5)
    cvf <- data.frame(matrix("",nrow=nr*5, ncol=0))
    
    for (b in 1:100){
        
        c.list = vector("list", 5)
        
        ctemp = sample(Y1$SN,5*nr)
        c.list[[b]] = ctemp
        cvf[b] <- cbind(c.list[[b]])
    }
    fold <- rep(1:5, times = nr)
    cvf <- cbind(fold,cvf)

    print(nrow(cvf))
    }


[1] 65
[1] 65
[1] 80
[1] 135
[1] 35


In [8]:
GBS=my.read.vcf(file= "data/SAP_all_taxa.vcf", header=TRUE, stringsAsFactors = TRUE, as.is=TRUE)


f.column <- grep("FORMAT", colnames(GBS))

##Function Parse vcf file to convert to -1,0,1 format
parse.GBS <- function(x) {
  unique.x <- unique(x)
  alleles <- setdiff(unique.x,union("H","N"))
  y <- rep(0,length(x))
  y[which(x==alleles[1])] <- -1
  y[which(x==alleles[2])] <- 1
  y[which(x=="N")] <- NA
  return(y)
}

X <- apply(GBS[, -c(1:f.column)],1,parse.GBS)

X[1:5,1:5]

-1,-1,-1,-1,-1
-1,-1,-1,-1,-1
-1,-1,-1,-1,1
-1,-1,-1,-1,-1
-1,-1,-1,-1,-1


,Subpopulation,Cluster,Race,Origin,DTA,PH,GN,GW,GY,FLH,PL,BL,SN
PI152651,Caudatum,4,0,NA,66,146.61,1286,27.32,43.95,97.00000,14.33333,57.95000,1
PI17548,Kafir,2,0,NA,66,214.06,1167,15.62,26.66,156.83333,22.83333,83.41667,2
PI24969,Durra,3,0,NA,80,182.06,1319,29.92,50.86,162.00000,13.33333,41.83333,3
PI329435,Mixed,1,0,NA,80,95.50,1388,15.68,30.47,65.83333,26.00000,72.58333,4
PI329440,Kafir,2,0,NA,69,93.61,2141,17.61,48.35,58.00000,23.83333,67.16667,5
PI34911,Caudatum,4,0,NA,87,179.67,1587,25.50,52.56,150.00000,14.33333,39.75000,6


In [63]:
setwd("/Users/ssapkot/Documents/Experiments/SAP_GS_PopStr/Results/Prediction_Results/WR_AR_SameTPsize/WR_Pred_Accu/")

for (a in 1:5){
    
    Y1 = Y[Y$Cluster == a,]
    nr = floor(nrow(Y1)/5)
    cvf <- data.frame(matrix("",nrow=nr*5, ncol=0))
    cluster <- Y1$Race[a]
    
    for (b in 1:100){
        
        c.list = vector("list", 5)
        
        ctemp = sample(Y1$SN,5*nr)
        c.list[[b]] = ctemp
        cvf[b] <- cbind(c.list[[b]])
    }
    fold <- rep(1:5, times = nr)
    print(head(cvf))
    
for (j in 5:12) {

  for (i in 1:101) {
  CV.fold <- paste("V",toString(i-1),sep='')
 
  if (CV.fold == "V0") {
    Total_Result <- c()
      result <- c()
  }
  else {  
    
  Z <- cvf[,CV.fold]
  Z <- sort(Z) ##sort randomly selected individuals by taxa order, and so the pheno and geno will be in the same order when subsetted
  
  X1 <- X[Z,]
  
  A <- A.mat(X1)
  
  rownames(A) <- 1:nrow(X1)
  P <- Y[Z,]
  cvs <- sample(fold)#fold is determined by folds assign while making the cvf dataframe 
  print(cvs)
  y = P[,j]
  col = names(P[j])
  
  yhat <- data.frame(cbind( y, yhat = 0))
  yhat$yhat <- as.numeric(yhat$yhat)
  row.names(yhat) <- row.names(y)
  
      corr <- c()
      var_x <- c()
      var_y <- c()
      cov_xy <- c()
  
      for (k in 1:5) {
    # Make training (TRN) and testing (TST) dfs
    tst <- which(cvs == k) ##cvs == whichever cluster/race is to be predicted
    yNA <- y
    yNA[tst] <- NA # Mask yields for validation set
    df <- data.frame(y=yNA,gid=1:nrow(A)) # Set up dataframe with traits and genotype labels (same order as in A1) 
    
    # Build rrBLUP model and save yhat for the masked values
    rrblup <- kin.blup(df,K=A,geno="gid",pheno="y") #optional parameters: fixed effects, gaussian kernel, covariates
    yhat$yhat[tst] = rrblup$pred[tst]
    }
    corr <- cor(yhat$y[tst],yhat$yhat[tst],use="complete")
    var_x <- var(yhat$yhat[tst], use="complete")
    var_y <- var(yhat$y[tst], use="complete")
    cov_xy <- cov(yhat$y[tst],yhat$yhat[tst], use="complete")
      
      result <- c(corr,var_x,var_y,cov_xy)
          
      }
      
      Total_Result <- cbind(Total_Result,result)
  
  }
    rownames(Total_Result) <- c("corr","var_x","var_y","cov_xy")
    
  write.csv(Total_Result, file = paste("WR_Corr-Cov_",a,"_",col,".csv", sep=""))
    }

}

   V1  V2  V3  V4  V5  V6  V7  V8  V9 V10 V11 V12 V13 V14 V15 V16 V17 V18 V19
1 204 367 203  23 188  51  76 279 351 137   4 242 189 198 198 252 164 165 136
2  65  49 341 188 268 189 170 378 189 196 229 367 118 380  68  47 324 196 158
3 118 227 198 242 385 204 204 127 188 341  42 199 158 189 385 165 129  32 170
4 202 196  77 341 233 142 288 229 233  62  45 279  42 105  42 105 170 215 198
5 164 142 324  45 197  65 278 203  38   4 341  77 288 188 203 189 242 279 269
6 199 218  47   4   4 178 165 158  42  68 126 217 127 268  51  51 189 351  61
  V20 V21 V22 V23 V24 V25 V26 V27 V28 V29 V30 V31 V32 V33 V34 V35 V36 V37 V38
1 380  61 278 269 233 146  94 136  61 351 164 351  32  61  47 218  68  76 217
2  42 127 233 160 158 165 164 198 137 188 197  69 278 204  68 136 279 170 105
3 241 269 242 105 146 208 165  23 385  45 160  38 203 197  23  38 118 164 385
4  38 378 367  45 269 187 137  68  32 197  77 188  45  23 197  65  47 158 160
5 136  65 118 136 164 367 367  76 199 105 203 252 208 146 165 18

In [ ]:
###When  writing the  file it would be useful to put in race and trait information,
###and also make a separate file/dataframe for mean and standard deviations
